#**Criando o Modelo REF (Reconhecimento de Emoções por Fala)**

Conectando ao drive para:
- Ter acesso a pasta de espectrogramas
- Salvar o modelo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Criação do modelo utilizando Resnet34

*Resnet34 é um modelo de classificação de imagens de última geração, estruturado como uma rede neural convolucional de 34 camadas e definido em " Deep Residual Learning for Image Recognition ". Restnet34 é pré-treinado no conjunto de dados ImageNet que contém mais de 100.000 imagens em 200 classes diferentes.*

*No entanto, RestNet é diferente das redes neurais tradicionais no sentido de que pega resíduos de cada camada e os utiliza nas camadas conectadas subsequentes (semelhante às redes neurais residuais usadas para previsão de texto).*

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

# Defina as classes de emoções
EMOTIONS = ["felicidade", "tristeza", "raiva", "neutro", "medo"]

# Crie uma classe personalizada para o conjunto de dados
class AudioEmotionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []  # Lista para armazenar pares (imagem, rótulo)

        # Carregar imagens e atribuir rótulos
        for i, emotion in enumerate(EMOTIONS):
            emotion_dir = os.path.join(root_dir, emotion)
            for filename in os.listdir(emotion_dir):
                image_path = os.path.join(emotion_dir, filename)
                self.data.append((image_path, i))  # (imagem, rótulo)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = plt.imread(image_path)  # Carregar imagem
        if self.transform:
            image = self.transform(image)
        return image, label

# Defina transformações de dados
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Crie conjuntos de treinamento e validação
train_dataset = AudioEmotionDataset("/content/drive/MyDrive/espectrogramas", transform=transform)

# Use train-test split para separar um subconjunto para validação
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Defina os dataloaders para treinamento e validação
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Crie o modelo ResNet-34 e defina a função de perda e otimizador
model = models.resnet34(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(EMOTIONS))  # Camada de saída com número de classes de emoções

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Treine o modelo
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        val_loss = 0.0
        predicted_labels = []
        true_labels = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                predicted_labels.extend(predicted.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")

    return model, true_labels, predicted_labels

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model, true_labels, predicted_labels = train_model(model, train_loader, val_loader, criterion, optimizer)

# Avalie o modelo
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=EMOTIONS))

# Matriz de confusão
confusion = confusion_matrix(true_labels, predicted_labels)
print("Matriz de Confusão:")
print(confusion)

# Salve o modelo treinado
torch.save(model.state_dict(), "/content/drive/My Drive/audio_emotion_model.pth")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 132MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:152: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This me

Epoch 1/10, Loss: 1.6725662776402064, Val Loss: 1.6553897857666016
Epoch 2/10, Loss: 1.5146723645074027, Val Loss: 1.598442256450653
Epoch 3/10, Loss: 1.4058803490230016, Val Loss: 1.5615379214286804
Epoch 4/10, Loss: 1.2562792982373918, Val Loss: 1.4866124987602234
Epoch 5/10, Loss: 1.1359678847449166, Val Loss: 1.4363253712654114
Epoch 6/10, Loss: 1.0224775416510445, Val Loss: 1.4235925078392029
Epoch 7/10, Loss: 0.8819666675158909, Val Loss: 1.372149407863617
Epoch 8/10, Loss: 0.7860389692442757, Val Loss: 1.359984815120697
Epoch 9/10, Loss: 0.6729327866009304, Val Loss: 1.3645009994506836
Epoch 10/10, Loss: 0.5345360807010106, Val Loss: 1.3286758661270142
Classification Report:
              precision    recall  f1-score   support

  felicidade       0.75      0.90      0.82        10
    tristeza       0.60      0.50      0.55        12
       raiva       0.59      0.77      0.67        13
      neutro       0.50      0.27      0.35        11
        medo       0.33      0.38     

--------------------------------------------------------------------------------

##Reconhecendo a emoção

Com o modelo criado, agora chegou a hora de testarmos o reconhecimento de emoções passando um áudio como referência.

*OBS.: O ideal é que seja um áudio fora do escopo de treinamento e validação do modelo*

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Verifique se a GPU está disponível e use-a para inferência, se possível
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define o dicionário de emoções
EMOTIONS = {
    0: "felicidade",
    1: "medo",
    2: "neutro",
    3: "raiva",
    4: "tristeza"
}

# Carregue o modelo treinado
model = models.resnet34(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(EMOTIONS))  # Ajuste o número de classes
model.load_state_dict(torch.load("/content/drive/My Drive/audio_emotion_model.pth", map_location=device))
model.to(device)
model.eval()

# Define a função de predição
def predict_emotion(audio_path):
    # Recebe um áudio
    audio, sampling_rate = librosa.load(audio_path)

    # Transforma o áudio em um espectrograma
    spectrogram = librosa.stft(audio, n_fft=512, hop_length=256)
    spectrogram_magnitude = np.abs(spectrogram)
    spectrogram_magnitude = spectrogram_magnitude.astype(np.float32)

    # Transforma o espectrograma em um tensor PyTorch
    spectrogram_tensor = torch.from_numpy(spectrogram_magnitude)
    spectrogram_tensor = spectrogram_tensor.unsqueeze(0).repeat(1, 3, 1, 1)

    # Move o tensor de entrada para a GPU
    spectrogram_tensor = spectrogram_tensor.to(device)

    # Faz a predição da emoção
    outputs = model(spectrogram_tensor)

    # Retorna a emoção predita
    _, predicted = torch.max(outputs, 1)
    predicted_emotion = EMOTIONS[predicted.item()]

    return predicted_emotion


# Exemplo de uso
predicted_emotion1 = predict_emotion("medo.wav")
predicted_emotion2 = predict_emotion("tristeza.wav")
predicted_emotion3 = predict_emotion("felicidade.wav")
predicted_emotion4 = predict_emotion("neutro.wav")
predicted_emotion5 = predict_emotion("raiva.wav")

print(predicted_emotion1)
print(predicted_emotion2)
print(predicted_emotion3)
print(predicted_emotion4)
print(predicted_emotion5)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


neutro
tristeza
neutro
neutro
raiva
